In [63]:
import pandas as pd

Filter and Prep Data

In [64]:
#1.

catalog = pd.read_csv(r"C:\Users\jnaid\demo_test\data_wrangling_304\data\assignment_5\course_catalog_by_major.csv")
pathways = pd.read_csv(r"C:\Users\jnaid\demo_test\data_wrangling_304\data\assignment_5\course_pathways.csv")

In [65]:
#2.

pathways_filtered = pathways[ (pathways.Program =='CECS') & (pathways.Major.isin(['Data science', 'Artificial intelligence', 'Cybersecurity'])) ]
pathways_filtered

,StudentID,Program,Major,CoursePath
80,S081,CECS,Data science,Intro to Python -> Statistics I -> Linear Alge...
81,S082,CECS,Data science,Intro to Python -> Statistics I -> Linear Alge...
82,S083,CECS,Data science,Intro to Python -> Statistics I -> Linear Alge...
83,S084,CECS,Data science,Intro to Python -> Statistics I -> Linear Alge...
84,S085,CECS,Data science,Intro to Python -> Statistics I -> Linear Alge...
...,...,...,...,...
195,S196,CECS,Cybersecurity,Intro to Cybersecurity -> Computer Networks ->...
196,S197,CECS,Cybersecurity,Intro to Cybersecurity -> Computer Networks ->...
197,S198,CECS,Cybersecurity,Intro to Cybersecurity -> Computer Networks ->...
198,S199,CECS,Cybersecurity,Intro to Cybersecurity -> Computer Networks ->...


In [66]:
#3.

pathways_filtered = pathways_filtered.copy() 
pathways_filtered.loc[:, 'CoursePath'] = pathways_filtered.CoursePath.str.split('->')#.apply(lambda x: ''.join(course.strip() for course in x))

pathways_filtered
pathways_filtered.loc[88]['CoursePath']


['Intro to Python ',
 ' Statistics I ',
 ' Linear Algebra ',
 ' Data Wrangling ',
 ' Data Visualization ',
 ' Machine Learning ',
 ' Big Data Analytics ',
 ' Database Systems ',
 ' Data Mining ',
 ' Data Ethics ',
 ' Recommender Systems ',
 ' Geospatial Analysis ',
 ' Visualization Tools ',
 ' Deep Learning ',
 ' Data Science Capstone 401']

In [ ]:
#4. 

#filters out rows with type 'elective' 
catalog_filtered = catalog[catalog['Type'] == 'Required']

def filter_required_courses(row):
    required = catalog_filtered[catalog_filtered['Major'] == row['Major']]['Course'].tolist()

    course_path_list = []
    for course in row['CoursePath']:
        if course in required:
            course_path_list.append(course)
    return course_path_list

pathways_filtered['CoursePath'] = pathways_filtered.apply(filter_required_courses, axis=1)

pathways_filtered
pathways_filtered.loc[88]['CoursePath']

['Intro to Python',
 'Statistics I',
 'Linear Algebra',
 'Data Wrangling',
 'Data Visualization',
 'Machine Learning',
 'Big Data Analytics',
 'Database Systems',
 'Data Mining',
 'Data Science Capstone 401']

2. Extract Transitions

In [87]:
transition_list = []


for row in pathways_filtered['CoursePath']:
    for i in range(len(row) - 1):
        first_course = row[i]
        next_course = row[i + 1]
        transition_list.append((first_course, next_course))

transition_list

[('Intro to Python', 'Statistics I'),
 ('Statistics I', 'Linear Algebra'),
 ('Linear Algebra', 'Data Wrangling'),
 ('Data Wrangling', 'Data Visualization'),
 ('Data Visualization', 'Machine Learning'),
 ('Machine Learning', 'Big Data Analytics'),
 ('Big Data Analytics', 'Database Systems'),
 ('Database Systems', 'Data Mining'),
 ('Data Mining', 'Data Science Capstone 401'),
 ('Intro to Python', 'Statistics I'),
 ('Statistics I', 'Linear Algebra'),
 ('Linear Algebra', 'Data Wrangling'),
 ('Data Wrangling', 'Data Visualization'),
 ('Data Visualization', 'Machine Learning'),
 ('Machine Learning', 'Big Data Analytics'),
 ('Big Data Analytics', 'Database Systems'),
 ('Database Systems', 'Data Mining'),
 ('Data Mining', 'Data Science Capstone 401'),
 ('Intro to Python', 'Statistics I'),
 ('Statistics I', 'Linear Algebra'),
 ('Linear Algebra', 'Data Wrangling'),
 ('Data Wrangling', 'Data Visualization'),
 ('Data Visualization', 'Machine Learning'),
 ('Machine Learning', 'Big Data Analytics'),

In [115]:
transition_df = pd.DataFrame(transition_list, columns=['First Course', 'Next Course'])
transition_df

,First Course,Next Course
0,Intro to Python,Statistics I
1,Statistics I,Linear Algebra
2,Linear Algebra,Data Wrangling
3,Data Wrangling,Data Visualization
4,Data Visualization,Machine Learning
...,...,...
1075,Network Security,System Security
1076,System Security,Ethical Hacking
1077,Ethical Hacking,Digital Forensics
1078,Digital Forensics,Security Policies


In [113]:
transition_count_df = (
    transition_df.groupby(['First Course', 'Next Course']).size().reset_index(name='Count')
)

transition_count_df

,First Course,Next Course,Count
0,Big Data Analytics,Database Systems,40
1,Computer Networks,Operating Systems,40
2,Computer Vision,Reinforcement Learning,40
3,Cryptography,Network Security,40
4,Data Mining,Data Science Capstone 401,40
5,Data Visualization,Machine Learning,40
6,Data Wrangling,Data Visualization,40
7,Database Systems,Data Mining,40
8,Deep Learning,Computer Vision,40
9,Digital Forensics,Security Policies,40


3. The Sankey Diagram

In [ ]:

first_courses = transition_count_df['First Course'].tolist()
next_courses = transition_count_df['Next Course'].tolist()

all_courses = []
for course in first_courses + next_courses:
    if course not in all_courses:
        all_courses.append(course)

label_to_index = {}
i = 0
for course in all_courses:
    label_to_index[course] = i
    i += 1


transition_count_df['first_index'] = transition_count_df['First Course'].map(label_to_index)
transition_count_df['next_index'] = transition_count_df['Next Course'].map(label_to_index)

#sankey
import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node=dict(label=list(label_to_index.keys())),
    link=dict(
        source=transition_count_df['first_index'],
        target=transition_count_df['next_index'],
        value=transition_count_df['Count']
    )
)])

fig.show()
transition_count_df

,First Course,Next Course,Count,first_index,next_index
0,Big Data Analytics,Database Systems,40,0,7
1,Computer Networks,Operating Systems,40,1,18
2,Computer Vision,Reinforcement Learning,40,2,20
3,Cryptography,Network Security,40,3,16
4,Data Mining,Data Science Capstone 401,40,4,24
5,Data Visualization,Machine Learning,40,5,15
6,Data Wrangling,Data Visualization,40,6,5
7,Database Systems,Data Mining,40,7,4
8,Deep Learning,Computer Vision,40,8,2
9,Digital Forensics,Security Policies,40,9,21


In [121]:
all_courses

['Big Data Analytics',
 'Computer Networks',
 'Computer Vision',
 'Cryptography',
 'Data Mining',
 'Data Visualization',
 'Data Wrangling',
 'Database Systems',
 'Deep Learning',
 'Digital Forensics',
 'Ethical Hacking',
 'Ethics in AI',
 'Intro to Cybersecurity',
 'Intro to Python',
 'Linear Algebra',
 'Machine Learning',
 'Network Security',
 'Neural Networks',
 'Operating Systems',
 'Probability & Statistics',
 'Reinforcement Learning',
 'Security Policies',
 'Statistics I',
 'System Security',
 'Data Science Capstone 401',
 'Artificial Intelligence Capstone 401',
 'Cybersecurity Capstone 401']